In [91]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [92]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import os
from dotenv import load_dotenv


In [93]:
load_dotenv()
api_key = os.getenv("ETHERSCAN_API_KEY")

True

In [94]:
# Example addresses
addresses = [
    "0x7568160091792b16066A06a115704b059814e9cc",
    "0xF58daEC3ABEe8b51F832b6bA3A06Fab2Aad3E0A8",
    "0xfd78EE919681417d192449715b2594ab58f5D002",
    "0xAbb07822F471773Ff00b9444308ceEB7cf0dACa7",
    "0xa42303EE9B2eC1DB7E2a86Ed6C24AF7E49E9e8B9"
]

In [95]:
# Known mixer/bridge contract addresses
known_services = {
    'TornadoCash': [
        '0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc',
        '0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936',
        '0x5efda50f22d34F262c29268506C5Fa42cB56A1Ce',
        '0x0836222F2B2B24A3F36f98668Ed8F0B38D1a872f',
        '0x169AD27A470D064DEDE56a2D3ff727986b15D52B',
    ],
    'Tokenlon': ['0x4a14347083b80e5216ca31350a2d21702ac3650d'],
    'UniswapV2': ['0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D'],
    '1inch': ['0x1111111254fb6c44bac0bed2854e76f90643097d'],
}

In [96]:
# Transaction fetching functions
def fetch_erc20_transactions(address):
    url = f"https://api.etherscan.io/api"
    params = {
        'module': 'account',
        'action': 'tokentx',
        'address': address,
        'startblock': 0,
        'endblock': 99999999,
        'sort': 'asc',
        'apikey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] == '1':
        df = pd.DataFrame(data['result'])
        df['token_value'] = df['value'].astype(float) / (10 ** df['tokenDecimal'].astype(int))
        df['time'] = pd.to_datetime(df['timeStamp'].astype(int), unit='s', utc=True)
        return df
    return pd.DataFrame()


In [97]:
def fetch_normal_transactions(address):
    url = f"https://api.etherscan.io/api"
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': address,
        'startblock': 0,
        'endblock': 99999999,
        'sort': 'asc',
        'apikey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] == '1':
        df = pd.DataFrame(data['result'])
        df['value_eth'] = df['value'].astype(float) / 1e18
        df['time'] = pd.to_datetime(df['timeStamp'].astype(int), unit='s', utc=True)
        return df
    return pd.DataFrame()

In [98]:
def analyze_address(address):
    output = f"\n--- Analyzing address: {address} ---\n"

    # Fetch transactions
    erc20 = fetch_erc20_transactions(address)
    normal = fetch_normal_transactions(address)

    # Add lowercase columns
    erc20['to_lower'] = erc20['to'].str.lower()
    erc20['from_lower'] = erc20['from'].str.lower()
    normal['to_lower'] = normal['to'].str.lower()
    normal['from_lower'] = normal['from'].str.lower()

    # Store plot data for later
    plots_to_show = []

    # Textual analysis
    for name, contract_list in known_services.items():
        addr_set = [a.lower() for a in contract_list]

        # ERC-20 txs
        erc20_hits = erc20[erc20['to_lower'].isin(addr_set) | erc20['from_lower'].isin(addr_set)]
        erc20_count = len(erc20_hits)
        erc20_volume = erc20_hits['token_value'].sum()

        # ETH txs
        normal_hits = normal[normal['to_lower'].isin(addr_set) | normal['from_lower'].isin(addr_set)]
        normal_count = len(normal_hits)
        normal_volume = normal_hits['value_eth'].sum()

        # Print analysis
        print(f"{name}:")
        print(f"  ERC-20 txs: {erc20_count} | Token volume: {erc20_volume:.2f}")
        print(f"  ETH txs  : {normal_count} | ETH volume: {normal_volume:.4f}")

        # Save dataframes for plotting
        if not erc20_hits.empty:
            plots_to_show.append(('ERC-20', name, address, erc20_hits.copy()))

        if not normal_hits.empty:
            plots_to_show.append(('ETH', name, address, normal_hits.copy()))

    return plots_to_show  # Only return plots, keep printing here




In [100]:
from collections import defaultdict

# Group charts by address
charts_by_address = defaultdict(list)

for tx_type, name, addr, df in all_plots:
    charts_by_address[addr].append((tx_type, name, df))

# Plot charts grouped under each address
for addr, charts in charts_by_address.items():
    print(f"\n📊 Charts for address: {addr}")
    for tx_type, service, df in charts:
        plt.figure(figsize=(10, 4))
        df.set_index('time').resample('D').size().plot(
            title=f"{addr} - {service} {tx_type} txs"
        )
        plt.ylabel(f"{tx_type} Tx Count")
        plt.tight_layout()
        plt.show()
